In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
mistral_checkpoint = "/kaggle/input/mistral/pytorch/7b-v0.1-hf/1"
mistral_tokenizer = AutoTokenizer.from_pretrained(mistral_checkpoint)
mistral_model = AutoModelForCausalLM.from_pretrained(
        mistral_checkpoint,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
    )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
t5_checkpoint = "t5-base"
t5_tokenizer = AutoTokenizer.from_pretrained(t5_checkpoint, max_length = 512)
t5_translator = pipeline("translation_en_to_de"
                         , model = t5_checkpoint
                         , clean_up_tokenization_spaces = True)

In [10]:
def get_gen_text(prompt, model, tokenizer):
    model_inputs = tokenizer([prompt], return_tensors="pt")
    generated_ids = model.generate(**model_inputs
                                   , max_new_tokens=100
                                   , pad_token_id = tokenizer.eos_token_id
                                   , do_sample=True)
    gen_text = tokenizer.batch_decode(generated_ids)[0]
    gen_list = gen_text.replace(prompt,"").replace("<s>","").split("\n")[:-1]
    stripped_list = [item.strip() for item in gen_list]
    
    return " ".join(stripped_list)

In [15]:
previous_occasion = "There is a restaraunt nearby the central station."
json = {"id":[], "translation":[]}
for i in range(1):
    prompt = f'''
                Describe an occasion in one short sentence.
                The occasion should differ from this one {previous_occasion}
                '''
    previous_occasion = get_gen_text(prompt, mistral_model, mistral_tokenizer)
    translation = t5_translator(f"translate English to German: {previous_occasion}")
    translation = translation[0]['translation_text']
    json["id"].append(i)
    json["translation"].append({"en":previous_occasion, "de":translation})
    print(i)

0


In [16]:
json

{'id': [0],
 'translation': [{'en': 'What do you do usually before you go? Do you go with anyone? How do you get there? How did you realize that everything went wrong? When did you realize? What happened? How and when did you change the situation? Did you have to ask someone? What helped you to go further?',
   'de': 'Was tun Sie normalerweise vor Ihrer Reise? Gehen Sie mit jemandem?'}]}